In [3]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, types
from sqlalchemy.pool import NullPool
from hubspot.crm.contacts import SimplePublicObjectInput, ApiException
from hubspot.auth.oauth import ApiException
from validate_email import validate_email
import hubspot
import os
from dotenv import load_dotenv

load_dotenv()
DATABASE_CONNECTION_URI = os.environ["DB_URL"]
ACCESS_TOKEN = os.environ["ACCESS_TOKEN"]
client_id = os.environ["CLIENT_ID"]
client_secret = os.environ["CLIENT_SECRET"]

# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

In [4]:
class HubspotAPI:
    def __init__(self):
        # API KEY
        self.access_token = ACCESS_TOKEN
        self.client = hubspot.Client.create(access_token = self.access_token)
        self.max_results = 1000000

    def raw_export_contacts(self, properties_dict):
        # Assistant Variables
        results = []
        after = 0

        while str(after).isnumeric() and len(results) < self.max_results:
            try:
                api_response = self.client.crm.contacts.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    associations=["deals", "companies"],
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Contacts Export has gathered " + str(len(results)) + " Contacts")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)

        try:
            df.to_sql('hs_contacts', con=engine, if_exists='replace', index=False)
            print(f"La tabla hs_contacts se actualizó correctamente en la base de datos.")
        except Exception as e:
            print(f"Error al intentar escribir en la base de datos: {str(e)}")

        assoc_results = []
        itterator = results.copy()
        for result in itterator:
            try:
                lists_to_append = result["associations"]["deals"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["contact_id"] = result["id"]
                    dict_to_append["deal_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            except:
                assoc_results.append([{"contact_id": result["id"], "deal_id": None}])
        contacts_id = []
        deal_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                contacts_id.append(item[i][a])
                deal_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"contact_id": contacts_id, "deals_id": deal_id})

        try:
            df.to_sql('hs_contacts_to_deals', con=engine, if_exists='replace', index=False)
            print(f"La tabla hs_contacts_to_deals se actualizó correctamente en la base de datos.")
        except Exception as e:
            print(f"Error al intentar escribir en la base de datos: {str(e)}")

        assoc_results = []
        for result in itterator:
            try:
                lists_to_append = result["associations"]["companies"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["contact_id"] = result["id"]
                    dict_to_append["company_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            except Exception as e:
                assoc_results.append([{"contact_id": result["id"], "company_id": None}])
        contacts_id = []
        companies_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                contacts_id.append(item[i][a])
                companies_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"contact_id": contacts_id, "company_id": companies_id})

        try:
            df.to_sql('hs_contacts_to_companies', con=engine, if_exists='replace', index=False)
            print(f"La tabla hs_contacts_to_companies se actualizó correctamente en la base de datos.")
        except Exception as e:
            print(f"Error al intentar escribir en la base de datos: {str(e)}")
        
    def raw_export_companies(self, properties_dict):
        # Parameters
        results = []
        after_results = []
        previous_after = None
        after = 0

        while str(after).isnumeric():
            try:
                api_response = self.client.crm.companies.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    associations=["deals"],
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Companies Export has gathered " + str(len(results)) + " Companies")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)

        try:
            df.to_sql('hs_companies', con=engine, if_exists='replace', index=False)
            print(f"La tabla hs_companies se actualizó correctamente en la base de datos.")
        except Exception as e:
            print(f"Error al intentar escribir en la base de datos: {str(e)}")
        
        assoc_results = []
        itterator = results.copy()
        for result in itterator:
            if result["associations"] is not None:
                lists_to_append = result["associations"]["deals"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["company_id"] = result["id"]
                    dict_to_append["deal_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            else:
                assoc_results.append([{"company_id": result["id"], "deal_id": None}])
        company_id = []
        deal_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                company_id.append(item[i][a])
                deal_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"company_id": company_id, "deals_id": deal_id})

        try:
            df.to_sql('hs_companies_to_deals', con=engine, if_exists='replace', index=False)
            print(f"La tabla hs_companies_to_deals se actualizó correctamente en la base de datos.")
        except Exception as e:
            print(f"Error al intentar escribir en la base de datos: {str(e)}")
        

    def raw_export_deals(self, properties_dict):
        # Assistant Variables
        results = []
        after_results = []
        previous_after = None
        after = 0
        while str(after).isnumeric():
            try:
                api_response = self.client.crm.deals.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Deals Export has gathered " + str(len(results)) + " Deals")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)

        # Dictionary to map old column names to new column names
        column_mapping = {
            'associated_company_proxy': 'associated_company',
            'associated_contact_proxy': 'associated_contact',
            'canal_de_representaci_n': 'canal_de_representacion',
            'hs_lastmodifieddate': 'last_modified',
            'hs_object_id': 'deal_id',
            'hubspot_owner_assigneddate': 'owner_assigneddate',
            'hubspot_owner_id': 'owner_id',
            'pa_s': 'pais',
            'procedencia': 'procedencia_del_lead',
            'hs_projected_amount': 'amount_weighted'
     
            # Add more mappings as needed
        }

        # Use the rename() method to rename columns
        df.rename(columns=column_mapping, inplace=True)

        # Reorder columns for better read
        df = df[[
            'createdate',
            'deal_id',
            'dealname',
            'dealstage',
            'amount',
            'closedate',
            'days_to_close',
            'closed_won_reason',
            'closed_lost_reason',
            'pais',
            'owner_id',
            'owner_assigneddate',
            'pipeline',
            'procedencia_del_lead',
            'canal_de_entrada',
            'canal_de_representacion',
            'associated_company',
            'associated_contact',
            'asignacion_a_preventa',
            'last_modified',
            'hs_analytics_source'
        ]]

        # Define diccionarios de reemplazo para columnas específicas
        reemplazo_dealstage = {
            "5101741": 'Identificación de Oportunidad (Ventas)',
            "d85ad2ee-ad84-48f3-a4e6-3c2752733963": 'Primer Contacto 10% (Ventas)',
            "750846": 'Demo Realizada 15% (Ventas)',
            "eb078c2a-ae3e-411a-9ce6-b8e5ca06dcd5": 'Presupuesto Enviado 20% (Ventas)',
            "d5608044-cac5-4e0c-9838-755be8adf652": 'Opt Avanzada / En Piloto 50% (Ventas)',
            "afcc2b30-8757-4d7e-923d-053d7db0205c": 'OC en Proceso / Cercano a Cierre 90% (Ventas)',
            "3d0d99a8-5791-421b-a1e5-6234c0844b65": 'Deal Won (Ventas)',
            "03a76306-0231-417d-9223-5bd0b12f63d5": 'Deal Lost (Ventas)',
            "7356165": 'Oportunidad de Baja (Ventas)',
            "1005088": 'Partners potenciales (Partners)',
            "2ad1f599-a06f-4456-bea4-e28739d0c6fb": 'Partner confirmado (NDA firmado) - Coordinar Onboarding (Partners)',
            "17487102": 'Capacitación comercial coordinada (Partners)',
            "2014582": 'Capacitación técnica coordinada (Partners)',
            "1611634": 'Capacitación comercial y técnica coordinada (Partners)',
            "32753101-a1a6-4c44-a789-82576c7b1505": 'Partner certificado - VAR (Partners)',
            "7025326": 'Partner certificado - Referral (Partners)',
            "2491428": 'Canales en Pausa (Partners)',
            "ff4aae15-0c00-4fa8-a677-39b35df7924f": 'No hay interés/No califica (Partners)',
            "cf839eef-4bbd-4368-804d-6cf60dabc03c": 'Contactos Identficados (Prospección)',
            "d9852484-edc7-4623-9cec-6a1639cff93f": 'Secuencia Enviada (Prospección)',
            "cd642987-1f60-48d1-9d83-4042e15d4be8": 'En Seguimiento / No Contesta (Prospección)',
            "33ba8152-823d-49ff-abfc-cfb9951374f3": 'Agendamiento de Reunión (Prospección)',
            "45411143": 'Prospección Exitosa (Prospección)',
            "583eddf6-eeaa-418d-91d0-8de9e28a2ec3": 'No califica (Prospección)',
            "201584": 'Potentially Good Fit - Sin contacto previo (Investor Pipeline)',
            "201583": 'Exploratory meeting (Investor Pipeline)',
            "8b586784-2f37-4604-89bd-36d088131e4c": 'Potencial interest in following (Investor Pipeline)',
            "bbbfa8fe-eea1-4d0a-8c15-3b322e777348": 'Potencial lead Investor (Investor Pipeline)',
            "493985da-75dc-4e5d-b103-f7be26d8cdf7": 'Due dilligence (Investor Pipeline)',
            "92c9806c-f2b2-40bc-9e08-2e50738b4050": 'No fit (Investor Pipeline)',
            "839d6147-109d-4a3e-853b-25ecd3c9124e": 'Stand by (Investor Pipeline)',
            "23780789": 'Clientes (Customer Success & Upsell)',
            "93563059": 'Reunion 1  (Customer Success & Upsell)',
            "17681683": 'Reunion 2 (Customer Success & Upsell)',
            "17681685": 'Reunion 3 (Customer Success & Upsell)',
            "138434867": 'En Capacitacion (Customer Success & Upsell)',
            "138434869": 'Upsell (Customer Success & Upsell)',
            "138434868": 'Fin del ciclo (Customer Success & Upsell)',
            "22687028": 'Derivado a Soporte (Customer Success & Upsell)',
            "17681687": 'Derivado a Comercial (Customer Success & Upsell)',
            "17681688": 'Derivado a Renovacion (Customer Success & Upsell)',
            "29561767": 'Sin Respuesta (Customer Success & Upsell)'
        }
        reemplazo_pipeline = {
            "7742d15e-56ae-4415-9c86-567ad766837a": 'Ventas',
            "820bd061-2228-4455-a379-4bf409d4554a": 'Partners',
            "1e666b90-c872-4a63-a4c1-1bce1dc79947": 'Prospección',
            "b1830386-2a88-458f-9cb4-71511efaaf87": 'Investor Pipeline',
            "5667166": 'Customer Success & Upsell'
        }
        reemplazo_deal_owner = {
            "7288359": 'Juan Martín Balan',
            "7332199": 'Nicolas Demner',
            "7857377": 'Martin Matias Fernandez Canto',
            "9194130": 'Joaquin Zoilo',
            "9430040": 'Gustavo Lauria',
            "30126325": 'Ventas Debmedia',
            "34868053": 'Matías Restahinoch',
            "38062801": 'Agustin Gelman',
            "38276053": 'Agustina Arroyo',
            "38627077": 'Tomas Noya',
            "55049136": 'Matias Pumo',
            "55049155": 'Camilo Varacalli',
            "56630584": 'Nicolas Menzaghi',
            "101057642": 'Roswel Amador',
            "112117634": 'Camila Acosta',
            "121481834": 'Soporte Debmedia',
            "246829367": 'Rafael Mattos',
            "312155919": 'Mariano Ahualli',
            "326472108": 'Lucía Imperiali',
            "366806335": 'Jaime Rodriguez',
            "396946262": 'valentina mineo',
            "399508303": 'Mateo Scapoli',
            "546914565": 'Iván Federico',
            "584399104": 'Aaron Escamilla',
            "597453098": 'Agustín Dimaio',
            "613110176": 'Pablo Prez',
            "653869216": 'Juan Lissarrague',
            "679733780": 'Agustina Coronel',
            "8231273": 'deactivated user',
            "24586352": 'deactivated user',
            "14224485": 'deactivated user'
        }
        # Agrega más diccionarios según tus necesidades

        # Realiza el reemplazo en las columnas específicas
        df['dealstage'] = df['dealstage'].replace(reemplazo_dealstage)
        df['pipeline'] = df['pipeline'].replace(reemplazo_pipeline)
        df['owner_id'] = df['owner_id'].replace(reemplazo_deal_owner)

        # Diccionario de países y regiones comerciales
        diccionario_region_comercial = {
            'Argentina': 'Región 1 - AR, BO, PY, UR',
            'Bolivia': 'Región 1 - AR, BO, PY, UR',
            'Paraguay': 'Región 1 - AR, BO, PY, UR',
            'Uruguay': 'Región 1 - AR, BO, PY, UR',
            'Chile': 'Región 2 - CL, PE',
            'Perú': 'Región 2 - CL, PE',
            'Peru': 'Región 2 - CL, PE',
            'Colombia': 'Región 3 - CO, EC',
            'Ecuador': 'Región 3 - CO, EC',
            'México': 'Región 4 - MX',
            'Mexico': 'Región 4 - MX',
            'Guatemala': 'Región 5 - Centroamérica',
            'Honduras': 'Región 5 - Centroamérica',
            'República Dominicana': 'Región 5 - Centroamérica',
            'Republica Dominicana': 'Región 5 - Centroamérica',
            'Guatemala': 'Región 5 - Centroamérica',
            'Puerto Rico': 'Región 5 - Centroamérica',
            'Nicaragua': 'Región 5 - Centroamérica',
            'Brasil': 'Región 6 - BR',
            'España': 'Región 7 - Europa',
            # Agrega más países según sea necesario
        }

        # Función para mapear la región comercial evitando asignar 'Otra Región' cuando el país es nulo
        def obtener_region_comercial(pais):
            if pais is not None and pais in diccionario_region_comercial:
                return diccionario_region_comercial[pais]
            else:
                return None  # O puedes devolver algún valor predeterminado según tus necesidades

        # Aplicar la función para crear la nueva columna "region_comercial"
        df['region_comercial'] = df['pais'].map(obtener_region_comercial)

        try:
            df.to_sql('hs_deals', con=engine, if_exists='replace', index=False)
            print(f"La tabla hs_deals se actualizó correctamente en la base de datos.")
        except Exception as e:
            print(f"Error al intentar escribir en la base de datos: {str(e)}")
        
def update_database():

    # Definir las propiedades de cada entidad de hubspot que queremos incluir en la tabla

    hs_contacts_properties = {

        # vienen por default vienen por default con diccionario vacio
        # "createdate": 1,
        # "email": 1,
        # "firstname": 1,
        # "hs_object_id": 1,
        # "lastmodifieddate": 1,
        # "lastname": 1,

        # vienen por default vienen por default con diccionario al menos una propiedad requerida
        # "createdate": 1,
        # "hs_object_id": 1,
        # "lastmodifieddate": 1,
        "email": 1,
        "firstname": 1,
        "lastname": 1,
        "hs_lifecyclestage_subscriber_date": 1,
        "hs_lifecyclestage_lead_date": 1,
        "hs_lifecyclestage_marketingqualifiedlead_date": 1,
        "hs_lifecyclestage_salesqualifiedlead_date": 1,
        "hs_lifecyclestage_lead_date": 1,
        "hs_lifecyclestage_opportunity_date": 1,
        "hs_lifecyclestage_customer_date": 1,
        # "": 1,
    }

    hs_companies_properties = {
        # vienen por default sin requerirlas:
        # "createdate": 1,
        # "domain": 1,
        # "hs_lastmodifieddate": 1,
        # "hs_object_id": 1,
        # "name": 1,
        # "": 1,  
    }
  
    hs_deals_properties = {
        # vienen por default sin requerirlas:
        # "hs_object_id": 1,
        # "createdate": 1,
        # "hs_lastmodifieddate": 1,
        "dealname": 1,
        "dealstage": 1,
        "amount": 1,
        "hs_projected_amount": 1,
        "days_to_close": 1,
        # "amount_in_home_currency": 1,
        "pa_s": 1,
        "hubspot_owner_id": 1,
        "hubspot_owner_assigneddate": 1,
        "pipeline": 1,
        "procedencia": 1,
        "canal_de_entrada": 1,
        "canal_de_representaci_n": 1,
        "closedate": 1,
        "closed_won_reason": 1,
        "closed_lost_reason": 1,
        "associated_company_proxy": 1,
        "associated_contact_proxy": 1,
        "asignacion_a_preventa": 1,
        "hs_analytics_source": 1
    }
    
    # se instancia la clase HuspotAPI
    h = HubspotAPI()

    # Hubspot Export
    #h.raw_export_contacts(hs_contacts_properties)
    h.raw_export_companies(hs_companies_properties)
    h.raw_export_deals(hs_deals_properties)

update_database()


Hubspot Companies Export has gathered 100 Companies
Hubspot Companies Export has gathered 200 Companies
Hubspot Companies Export has gathered 300 Companies
Hubspot Companies Export has gathered 400 Companies
Hubspot Companies Export has gathered 500 Companies
Hubspot Companies Export has gathered 600 Companies
Hubspot Companies Export has gathered 700 Companies
Hubspot Companies Export has gathered 800 Companies
Hubspot Companies Export has gathered 900 Companies
Hubspot Companies Export has gathered 1000 Companies
Hubspot Companies Export has gathered 1100 Companies
Hubspot Companies Export has gathered 1200 Companies
Hubspot Companies Export has gathered 1300 Companies
Hubspot Companies Export has gathered 1400 Companies
Hubspot Companies Export has gathered 1500 Companies
Hubspot Companies Export has gathered 1600 Companies
Hubspot Companies Export has gathered 1700 Companies
Hubspot Companies Export has gathered 1800 Companies
Hubspot Companies Export has gathered 1900 Companies
Hu